<a href="https://colab.research.google.com/github/ksydata/PseudonymizeData_toPySQL/blob/main/DSwithSQL_SY_231220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os
import pandas as pd
from typing import *

# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format

Mounted at /content/drive


In [2]:
class GoogleDriveDataLoad:
  def __init__(self):
    self.file_path = None
    self.file_name = None

  def loadData(#self,
               file_path: str, file_name: str, encoder: str, id: int) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(file_path, file_name), encoding = encoder, index_col = id)

In [3]:
load = GoogleDriveDataLoad

In [4]:
DATA_LIST: Dict = {}
FILE_LIST = os.listdir("/content/drive/MyDrive/DataArchive/data_sample/")

In [5]:
for index, file in zip(list(range(0, 7, 1)), FILE_LIST):
  data = load.loadData(
      file_path = "/content/drive/MyDrive/DataArchive/data_sample/",
      file_name = file,
      encoder = "utf-8",
      id = 0)
  DATA_LIST[file] = data

In [ ]:
for index in range(0, 7, 1):
  print("\n", FILE_LIST[index])
  DATA_LIST[FILE_LIST[index]].info()


 실무자_통신1_v2.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   나이       100000 non-null  int64 
 1   이름       100000 non-null  object
 2   성별       100000 non-null  object
 3   우편번호     100000 non-null  int64 
 4   상세주소     100000 non-null  object
 5   핸드폰번호    100000 non-null  object
 6   업종명      100000 non-null  object
 7   가입상품     100000 non-null  object
 8   요금       100000 non-null  int64 
 9   가입일      100000 non-null  int64 
 10  회원유형     100000 non-null  object
 11  블랙리스트여부  100000 non-null  object
 12  연체횟수     100000 non-null  int64 
 13  멤버쉽등급    100000 non-null  object
 14  콘텐츠구매여부  100000 non-null  object
 15  구매금액     100000 non-null  int64 
 16  부가서비스    100000 non-null  object
dtypes: int64(6), object(11)
memory usage: 13.7+ MB

 실무자_외부결합2_교통카드정보_v1.csv
<class 'pandas.core.frame.DataFrame'>
Int64Inde

### 탐색 > 프로파일링 > 정제 > 셰이핑 > 분석
* 탐색

  데이터와 관련된 주제는 무엇인지, 데이터가 어디에서 생성되었는지, 저장된 데이터베이스테이블은 무엇인지 등을 자세히 알아가는 과정

* 프로파일링

  데이터셋에서 고유한 값은 무엇이며, 값의 분포는 어떠한지 등을 확인하는 과정

* 정제

  틀리거나 불완전한 데이터를 수정하고 카테고리를 추가하거나 결측값 등을 다루는 과정

* 셰이핑

  데이터를 분석 목적에 따라 행과 열로 저장하는 과정

* 분석

  트렌드를 파악하고 결론을 내린 후 인사이트를 찾아내기 위해 데이터를 검토하는 과정

In [ ]:
CREDITDATA = DATA_LIST[FILE_LIST[5]]

신용카드 발급 건수별 고객 수 분포를 확인하는 쿼리

* 고객번호 컬럼
* 카드번호 컬럼

In [ ]:
"""
SELECT NUM_CREDITCARD_ISSUANCES, COUNT(*) as NUM_CONSUMERDS
FROM (
  SELECT CONSUMER_ID, COUNT(CREDITCARD_ID)
  FROM CREDITDATA
  GROUP BY 1
) subquery
GROUP BY 1
"""

In [ ]:
CREDITDATA[["신용카드월사용금액", "신용카드월현금서비스금액", "대출총금액"]].describe()

,신용카드월사용금액,신용카드월현금서비스금액,대출총금액
count,95000.0000,95000.0000,95000.0000
mean,5055081.6847,4997600.4163,2528637.7354
std,2855950.6290,2891598.6753,8848861.3134
min,100018.0000,34.0000,0.0000
25%,2587585.0000,2490506.5000,0.0000
50%,5063748.0000,4990128.0000,0.0000
75%,7525220.7500,7506426.5000,0.0000
max,9999798.0000,9999926.0000,49999255.0000


조건로직으로 구간화: 대출총금액이나 신용카드월사용액, 신용카드월현금서비스금액 등을 기준으로 고객의 여신거래규모를 구분함.

In [ ]:
"""
SELECT
    CASE WHEN AMT_CREDITCARD_PAYMENT <= 2500000 then "up to 2,500,"
    WHEN AMT_CREDITCARD_PAYMENT <= 5000000 then "2,500, - 5,000,"
    WHEN AMT_CREDITCARD_PAYMENT <= 7500000 then "5,000,- 7,500,"
    ELSE "7,500,+" END AS AMT_CREDITCARD_BIN,

    CASE WHEN AMT_CREDITCARD_PAYMENT <= 2500000 then "4th BOUND,"
    WHEN AMT_CREDITCARD_PAYMENT <= 5000000 then "3rd BOUND,"
    WHEN AMT_CREDITCARD_PAYMENT <= 7500000 then "2nd BOUND"
    ELSE "1st BOUND" END AS AMT_CREDITCARD_CATEGORY,

FROM CREDITDATA
GROUP BY 1, 2
"""

균등한 구간 범위 설정에 로그를 사용하기도 함. 특히 가장 작은 값과 큰 값의 차이가 큰 경우(웹사이트 방문자 분포, 자산 분포)

백분위수와 표준편차 활용: ntile() 윈도우 함수로 각 행의 N분위수 값을 계산하고 외부쿼리에서 구간별 범위 확인


In [ ]:
"""
SELECT
    ntile,
    MIN(AMT_CREDITCARD_PAYMENT) as LOWER_BOUND,
    MAX(AMT_CREDITCARD_PAYMENT) as UPPER_BOUND,
    COUNT(CREDITCARD_ID) as CNT_CREDITCARD
FROM
(
  SELECT
    CASE WHEN


)
"""
# ntile(NUM_BINS) over(PARTITION BY __ ORDER BY __)

### 시계열 분석

* 모든 사용자가 동이한 시간대 내 지역에 산다면 전혀 문제가 되지 않지만, 전 세계에 흩어져 있다면 DB에 저장된 UTC(Coordinated Universal Time)을 각 사용자의 로컬 시간에 맞게 변환해 분석해야 한다.

* 즉, UTC를 사용하면 데이터에 로그 또는 이벤트가 발생한 로컬 시간을 확인할 수 없다는 단점이 있다.

In [23]:
DATA_LIST[FILE_LIST[0]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   나이       100000 non-null  int64 
 1   이름       100000 non-null  object
 2   성별       100000 non-null  object
 3   우편번호     100000 non-null  int64 
 4   상세주소     100000 non-null  object
 5   핸드폰번호    100000 non-null  object
 6   업종명      100000 non-null  object
 7   가입상품     100000 non-null  object
 8   요금       100000 non-null  int64 
 9   가입일      100000 non-null  int64 
 10  회원유형     100000 non-null  object
 11  블랙리스트여부  100000 non-null  object
 12  연체횟수     100000 non-null  int64 
 13  멤버쉽등급    100000 non-null  object
 14  콘텐츠구매여부  100000 non-null  object
 15  구매금액     100000 non-null  int64 
 16  부가서비스    100000 non-null  object
dtypes: int64(6), object(11)
memory usage: 13.7+ MB


In [ ]:
# for column in ["업종명", "가입상품", "회원유형", "멤버쉽등급", "부가서비스"]:
  # print("\n", column, Counter(DATA_LIST[FILE_LIST[0]][column]))

In [27]:
Counter(DATA_LIST[FILE_LIST[0]]["가입상품"])

Counter({'모두안심 기가': 5862,
         '기가인터넷멀티 와이파이 1.7G': 5748,
         '기가프리미엄X5': 5808,
         '원격케어 기가라이트': 5768,
         '광랜인터넷 와이파이': 5840,
         '모두안심 광랜': 5779,
         '기가라이트인터넷 와이파이': 5914,
         '원격케어 광랜': 6063,
         '기가인터넷 와이파이': 5874,
         '기가프리미엄X2.5': 5926,
         '기가인터넷멀티': 5957,
         '기가프리미엄X10': 5949,
         '광랜인터넷': 5998,
         '원격케어 기가': 5732,
         '기가라이트인터넷': 6010,
         '기가인터넷': 5902,
         '모두안심 기가라이트': 5870})

In [28]:
Counter(DATA_LIST[FILE_LIST[0]]["멤버쉽등급"])

Counter({'일반': 38722,
         'WHITE': 32006,
         'VIP': 5929,
         'VVIP': 2991,
         'GOLD': 8709,
         'SILVER': 11643})

In [ ]:
"""
SELECT
  date_part("year", SUBSCRIPTION_DATE) AS SUBSCRIPTION_YEAR,
  SUBSCRIPTION_SERICE,
  MEMBERSHIP_TYPE,
  SUM(SUBSCRIPTION_FEE) AS SUBSCIRIPTION_SALES
FROM DATA_MOBILE_COMMUNICAION
WHERE SUBSCRIPTION_SERVICE IN ("모두안심 기가", "모두안심 광랜", "모두안심 기가라이트"),
AND MEMBERSHIP_TYPE IN ("일반", "WHITE", "SILVER")
GROUP BY 1, 2
ORDER BY 1, 2
"""